To Do:


* finish coding conditional FP Tree Builder
* Implement Association Pattern mining (just lookup support values to compute conf level)

## Tree Class and Methods

In [1]:
### Basic generic tree class
### Nodes have: a count, and children pointers (add_children automatically construct parent pointers)

class Node(object):
    def __init__(self, data):
        self.data = data
        self.count = 1
        self.children = []
        self.parent=[]

    def add_child(self, obj):
        obj.parent = self
        self.children.append(obj)
        
        
        
        
### Unpacking function to check counts (partially verify tree structure):
# Could roll this into a verification function later
def unpack(root_node, node_list = []):
    for node in root_node.children:
        node_list.append([node.data, node.count])
        unpack(node, node_list)
    return node_list




### Building Conditional FP trees

# Get leaves (bottom nodes)
def get_leafs(root_node, node_list = []):
    if root_node.children == []:
        node_list.append(root_node)
    else:
        for node in root_node.children:
            get_leafs(node, node_list)
    return node_list


### create tree from leaves (back traversal)

def grow_skeleton(node, tree_skeleton = []):
    tree_skeleton.append(node) 
    if node.parent == []:
        tree_skeleton.reverse()
        return tree_skeleton # so final skeleton goes from root to leaf
    else:
        grow_skeleton(node.parent, tree_skeleton)
        #tree_skeleton.reverse()
        return tree_skeleton



In [2]:
### Testing Block
# n = Node("A")
# p = Node("B")
# q = Node(7)
# r = Node(8)
# s = Node("C")
# n.add_child(p)
# n.add_child(r)
# L = [p,q,r,s]
#
# for node in L:
#     if node in n.children:
#         node.count += 1
#     else:
#         n.add_child(node)
#    
# for c in n.children:
#     print (c.data, c.count)
#   
#
# if p.data in [y.data for y in n.children]:
#     print("yes")
#   
#   
# print( n.children[2].children)
# n.children[2].add_child(q) 
# print("and now")
# print( n.children[2].children[0].data)



# ### Testing grow_skeleton():
#
# X = Node("X")
# Y = Node("Y")
# Z = Node("Z")
#
# X.add_child(Z)
#
# print(X.children[0].data)
# print(Z.parent.data)
# print([x.data for x in grow_skeleton(Z)])

# ### TEST BLOCK
#
# leafs = get_leafs(root_node)
#
# [x.data for x in leafs]
#
# leaf0 = leafs[0]
# leaf1 = leafs[1]
#
# print(leaf0 == leaf1, grow_skeleton(leaf0,[]) == grow_skeleton(leaf1,[]))
#
# print([x.data for x in grow_skeleton(leaf0,[])])
# print([x.data for x in grow_skeleton(leaf1,[])])
#
# len(grow_skeleton(leaf1,[]))


## Handle Data

In [3]:
import pandas as pd

### Sample Dataset

TDB = pd.read_excel("FoodMart.xls")

TDB.head()

,Acetominifen,Anchovies,Aspirin,Auto Magazines,Bagels,Batteries,Beer,Bologna,Candles,Canned Fruit,...,Sunglasses,Tofu,Toilet Brushes,Tools,Toothbrushes,Tuna,TV Dinner,Waffles,Wine,Yogurt
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# populate a dictionary of total purchases for each item
products = {}

for column in list(TDB):
    products[column] = sum(TDB[column])
    
# need to find a reasonable threshold (support)
import numpy as np
print("supp approx: " + str(float(np.mean([x[1] for x in products.items()])/len(TDB))))

# suggests a support threshold of ~ .033 (lowered it slightly since skewed data) or a count size of ~70
supp_threshold = 70

# get only items above threshold:
purchase_totals_above_threshold = [x for x in products.items() if x[1] > supp_threshold]
purchase_totals_above_threshold.sort(key=lambda x: x[1], reverse=True) # sorting
purchase_totals_above_threshold

###

# create new TDB with ordered max to min transactions in each row
sorted_columns = [x[0] for x in purchase_totals_above_threshold]

updated_TDB = TDB.reindex_axis(sorted_columns, axis=1, copy=True, method=None) 
# method=none ie drop columns that aren't in the new labels sorted_columns
# copy=true return a new object

updated_TDB.head()    

# print("TOTAL COUNTS FOR FREQUENT 1-ITEMSETS: ")
# print(purchase_totals_above_threshold)

supp approx: 0.03987306064880113


,Cheese,Soup,Dried Fruit,Cookies,Preserves,Wine,Frozen Vegetables,Canned Vegetables,Nuts,Milk,...,Jam,Pot Cleaners,Hamburger,Rice,Cleaners,Ice Cream,Plastic Utensils,Donuts,Tuna,Cold Remedies
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [13]:
### Build Tree
# read data from updated_TDB into a tree
root_node = Node('')
node_list = [root_node] # node list for debug purpose

import time

start_time = time.time()

for i,row in updated_TDB.iterrows():
    parent_node = root_node
    for j in range(len(row)):
        if row[j] > 0:
            # print('yes')
            name = sorted_columns[j]
            if name in [y.data for y in parent_node.children]:
                index = [y.data for y in parent_node.children].index(name)
                parent_node.children[index].count += 1
                parent_node = parent_node.children[index]
                
            else:
                x = Node(name)
                parent_node.add_child(x)
                parent_node = x
        # else:   print('no')
    if j % 500 == 0: print('done 500')
    
stop_time = time.time()

print("took " + str(stop_time-start_time))

# root_node now stores pointers to it's children, 
# storing information about the entire tree


# debug code
# #[print(x.data, x.children) for x in node_list]
# print([(node.data, node.count) for node in root_node.children])

took 1.2478680610656738


In [14]:
# ### Verify Tree
#
# tree = unpack(root_node)
# 
# ### compress node counts for same product
# 
# summary = {}
# for item in tree:
#     if item[0] not in summary:
#         summary[item[0]] = item[1] 
#     else:
#         summary[item[0]] += item[1]
#
# all_nodes = sorted(summary.items(), key=lambda x: x[1], reverse = True) 
#
# # all_nodes == purchase_totals_above_threshold # returns an error
# # inspection shows only flags false on a few items with same purchase counts:
# [ (x,y, x==y) for x,y in zip(all_nodes,purchase_totals_above_threshold) ] 
#
# ### Correct counts confirmed

In [15]:
### Build Pattern Bases

# Get leaves (bottom nodes)
leafs = get_leafs(root_node)

# build pattern bases
leafs = get_leafs(root_node)
conditional_tree_list = []

for leaf in leafs:
    # create an entry for pattern base with a leaf name-label and a count
    conditional_tree_list.append([leaf.data, grow_skeleton(leaf,[]), leaf.count])    

# checking:
# for subtree in conditional_tree_list:
#     print(subtree[0], [x.data for x in subtree[1]])

In [31]:
##### Build conditional FP tree

# recall supp_threshold = 70
min_support = 70

# get unique leaf names
leaf_names = list(set([leaf.data for leaf in leafs]))


# get pattern bases for leafs with same value (name) together in a list
# will use this list to build the conditional FP Trees
pattern_base_list = [1]*len(leaf_names)

for i in range(len(leaf_names)):
    leaf = leaf_names[i]
    temp_list = []
    for subtree in conditional_tree_list:
        subtree_leaf_name = subtree[0]
        if subtree_leaf_name == leaf:
            ##### put into their own area...
            temp_list.append([subtree[1], subtree[2]])
    pattern_base_list[i] = [leaf, temp_list]

# pattern_base_list

## Example Output
[(x[0], [y.data for y in x[1][2][0]]) for x in pattern_base_list][0:4]

# from itertools import combinations # to get combinations


[('Rice', ['', 'Cheese', 'Pizza', 'Coffee', 'Popcorn', 'Hard Candy', 'Rice']),
 ('Flavored Drinks', ['', 'Cheese', 'Canned Vegetables', 'Flavored Drinks']),
 ('Ice Cream', ['', 'Cheese', 'Bologna', 'Deli Salads', 'Ice Cream']),
 ('Pizza', ['', 'Soup', 'Chocolate Candy', 'Pizza'])]